In [1]:
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import h5py

import pandas as pd
import pprint

import pymongo
from pymongo import MongoClient

import requests

import databroker
from databroker import list_configs
from databroker import Broker

# make graphics inline
%matplotlib inline

from archiver import *

## Archiver Reader Configuration

In [2]:
# CSX Archiver
ar_config_csx = {'name' : 'csx', 'url' : 'http://xf23id-ca.cs.nsls2.local:17668', 
                 'timezone' : 'US/Eastern', 
                 'pvs' : {'pv1':'XF:23ID-ID{BPM}Val:PosXS-I'}}
ar_csx = ArchiverReader.from_config(ar_config_csx)

In [3]:
# Accelerator Archiver
ar_config_acc = {'name' : 'accelerator', 'url' : 'http://arcapp01.cs.nsls2.local:17668', 
                 'timezone' : 'US/Eastern', 
                 'pvs' : {'test_watT' :'UT:SB1-Cu:1{}T:Prmry-I'}}
ar_acc = ArchiverReader.from_config(ar_config_acc)

## Integration with Databroker (e.g., within NSLS-II configure_base)

In [4]:
db = Broker.named('csx')

for key in ArchiverReader.configs.keys():
    aes = ArchiverReader.from_config(ArchiverReader.configs[key])
    db.add_event_source(aes)
    
len(db.event_sources)

3

## Data Retrieval based on the Databroker Interface

In [5]:
# select header
hdr = db[69209]

In [6]:
# check stream names
db.stream_names_given_header(hdr)

['baseline', 'primary', 'archiver_test_watT', 'archiver_pv1']

In [7]:
# get table for selected pv
stream_name = 'archiver_pv1'
df = db.get_table(hdr, stream_name)
len(df)

660

In [8]:
df.head()

,time,archiver_pv1
seq_num,,
1,2016-10-06 21:54:28.966,0.004190
2,2016-10-06 21:54:29.066,0.004188
3,2016-10-06 21:54:29.166,0.004192
4,2016-10-06 21:54:29.266,0.004192
5,2016-10-06 21:54:29.366,0.004202


In [9]:
# get table for selected pv
stream_name = 'archiver_test_watT'
df = db.get_table(hdr, stream_name)
len(df)

3

In [10]:
df.head()

,time,archiver_test_watT
seq_num,,
1,2016-10-06 21:53:57.425,80.3
2,2016-10-06 21:55:07.431,80.4
3,2016-10-06 21:55:09.427,80.3


## Data Retrieval using stand-alone ArchiverReaders

In [11]:
# define timestamps
since  = hdr['start']['time']
until  = hdr['stop']['time'] + 10*(hdr['stop']['time'] - since)

In [12]:
# get the csx pv tables for given time interval
ar_csx  = ArchiverReader.named('csx')
dfs_csx = ar_csx.tables_given_times(since, until)
df = dfs_csx['pv1']
len(df)

7326

In [13]:
# get the accelerator pv tables for given time interval
ar_acc = ArchiverReader.named('accelerator')
dfs_acc = ar_acc.tables_given_times(since, until)
df = dfs_acc['test_watT']
len(df)

53